In [3]:
import pandas as pd
import numpy as np
import random
import gensim
from gensim.utils import simple_preprocess, tokenize
from gensim.parsing.preprocessing import STOPWORDS
import gensim.corpora as corpora
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.corpus import stopwords
import string 
from tqdm import tqdm
from pprint import pprint
import time
np.random.seed(69)
random.seed(69)

In [4]:
data=pd.read_csv("FAOLEX_All.csv")
# print(data.keys())
# print(df.info())
# print(keywords[:5])

# Create simplified version of the dataframe by discarding irrelevant columns 
to_drop = ['Record Id', 
           'Record URL', 
           'Document URL', 
           'Title',
           'Original title',
           'Available website',
           'Abstract']

data.drop(to_drop, inplace=True, axis=1)
# Remove NaN values in the Countries/Territory and Keywords attributes 
data_no_nan = data.dropna(subset=["Country/Territory", "Keywords"])
# Remove all documents that have been repealed (Repealed == Y)
data_no_repealed = data_no_nan[data_no_nan.Repealed != 'Y']

data_kw = data_no_repealed[["Country/Territory","Keywords"]]
data_kw['index'] = data_kw.index
keywords = data_kw

#preproccessing
tokenizer = RegexpTokenizer(r'\w+') # create tokenizer to tokenize docs 
stop_words = stopwords.words('english') # create list of stopwords
p_stemmer = PorterStemmer() # create stemmer instance 

doc_set = keywords["Keywords"].tolist()
processed_kw_list = []

# loop through document list
for i in tqdm(doc_set):
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in stop_words]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    processed_kw_list.append(stemmed_tokens)
    #processed_kw_list.append(stopped_tokens)
    
country = keywords["Country/Territory"].tolist()
    

C:\Users\Jacob pc\AppData\Local\Temp\ipykernel_6732\1787380598.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_kw['index'] = data_kw.index
100%|██████████| 172392/172392 [00:51<00:00, 3327.81it/s]


In [5]:
# Mapping from word IDs to words
# https://medium.com/analytics-vidhya/topic-modeling-using-gensim-lda-in-python-48eaa2344920
id2word = corpora.Dictionary(processed_kw_list)

# Prepare Document-Term matrix
corpus = []
for doc in processed_kw_list:
    corpus.append(id2word.doc2bow(doc))

In [6]:
# Load models
from top2vec import Top2Vec
from gensim import  models
top2vec_model = Top2Vec.load("top2vec_model")
lda_model = models.ldamodel.LdaModel.load("lda_model")
n_topics=100

C:\Users\Jacob pc\anaconda3\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Jacob pc\anaconda3\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Jacob pc\anaconda3\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopyth

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
from sklearn.neighbors import KNeighborsClassifier

In [9]:
feature_vectors_top2vec = []
feature_vectors_lda = []
country_all = []
for i in (range(len(country))):
    if len(country[i].split("; "))>=2:
        for j in range(len(country[i].split("; "))):
            #top2vec
            topic_nums,topic_score,_,_ = top2vec_model.get_documents_topics([i], reduced=False, num_topics=top2vec_model.get_num_topics())
            feature_vectors_top2vec.append(topic_score[0][topic_nums[0].argsort()])
            #LDA
            topic_probs = lda_model.get_document_topics(corpus[i], minimum_probability=0)
            probs = [t[-1] for t in topic_probs]
            feature_vectors_lda.append(probs)
            #
            country_all.append(country[i].split("; ")[j])
    else:
        #top2ved
        topic_nums,topic_score,_,_ = top2vec_model.get_documents_topics([i], reduced=False, num_topics=top2vec_model.get_num_topics())
        feature_vectors_top2vec.append(topic_score[0][topic_nums[0].argsort()])
        #LDA
        topic_probs = lda_model.get_document_topics(corpus[i], minimum_probability=0)
        probs = [t[-1] for t in topic_probs]
        feature_vectors_lda.append(probs)
        #
        country_all.append(country[i])
labels=country_all


In [11]:
from collections import Counter

In [12]:
# Stratified CV: 
X_top2vec = np.array(feature_vectors_top2vec)
X_lda = np.array(feature_vectors_lda)
y = np.array(labels)
neigh_top2vec = KNeighborsClassifier(n_neighbors=5)
neigh_lda = KNeighborsClassifier(n_neighbors=5)
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=11)
accu_stratified_lda = []
accu_stratified_top2vec = []
accu_stratified_baseline = []
for train_index, test_index in kfold.split(X_top2vec, y):
    x_train_top2vec, x_test_top2vec = X_top2vec[train_index], X_top2vec[test_index]
    x_train_lda, x_test_lda = X_lda[train_index], X_lda[test_index]
    y_train, y_test = y[train_index], y[test_index]
    neigh_top2vec.fit(x_train_top2vec, y_train)
    neigh_lda.fit(x_train_lda, y_train)
    accu_stratified_top2vec.append(metrics.accuracy_score(y_test, neigh_top2vec.predict(x_test_top2vec)))
    accu_stratified_lda.append(metrics.accuracy_score(y_test, neigh_lda.predict(x_test_lda)))
    # Baseline
    counter = Counter(y_train)
    most_common_element, occurrence_count = counter.most_common(1)[0]
    accu_stratified_baseline.append(occurrence_count/len(feature_vectors_lda))

C:\Users\Jacob pc\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


In [13]:
print("mean accuracy lda", np.mean(accu_stratified_lda))
print("mean accuracy top2vec", np.mean(accu_stratified_top2vec))
print("Baseline accuracy", np.mean(accu_stratified_baseline))


from scipy import stats

t_statistic, p_value = stats.ttest_rel(accu_stratified_lda, accu_stratified_top2vec)

# Print the test statistic and p-value
print("Paired t-test:")
print("Test statistic:", t_statistic)
print("p-value:", p_value)

mean accuracy lda 0.21993195119154513
mean accuracy top2vec 0.18370676267392294
Baseline accuracy 0.03838000435707942
Paired t-test:
Test statistic: 27.974258286262817
p-value: 0.00010027685978436129


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature_vectors_top2vec, labels, test_size = 0.3)

In [ ]:
neigh1 = KNeighborsClassifier(n_neighbors=5)
neigh1.fit(X_train, y_train)
y_pred_knn = neigh1.predict(X_test)
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred_knn))

Bedste n_neighbors - top2vec: 1(bedst 1.), 5(3.), 8(2.), 15(lidt dårligere) 25 (meget dårligere)
Bedste n_neighbors - lda: 1 (19.6), 3 ( 17.38) 5(18), 25 (17.6)

In [ ]:
# Mcnemars test
from scipy.stats import beta
from scipy.stats import binom
from sklearn.model_selection import ShuffleSplit
split_shuffle = ShuffleSplit(n_splits=1, random_state=11, test_size=0.3)
X_top2vec = np.array(feature_vectors_top2vec)
X_lda = np.array(feature_vectors_lda)
y = np.array(labels)
for train_index, test_index in split_shuffle.split(X_top2vec, y):
    x_train_top2vec, x_test_top2vec = X_top2vec[train_index], X_top2vec[test_index]
    x_train_lda, x_test_lda = X_lda[train_index], X_lda[test_index]
    y_train, y_test = y[train_index], y[test_index]

    d1_n_11=[]
    d1_n_10=[]
    d1_n_01=[]
    d1_n_00=[]

    d2_n_11=[]
    d2_n_10=[]
    d2_n_01=[]
    d2_n_00=[]

    d3_n_11=[]
    d3_n_10=[]
    d3_n_01=[]
    d3_n_00=[]

    lda_score=0
    top2vec_score=0
    base_score=0
    i=0
    # top2vec
    neigh_top2vec = KNeighborsClassifier(n_neighbors=1)
    neigh_top2vec.fit(x_train_top2vec, y_train)
    top2vec_score=neigh_top2vec.predict(x_test_top2vec) == y_test
    print("ACCURACY OF THE top2vec MODEL: ", metrics.accuracy_score(y_test, neigh_top2vec.predict(x_test_top2vec)))
    #LDA
    neigh_lda = KNeighborsClassifier(n_neighbors=1)
    neigh_lda.fit(x_train_lda, y_train)
    lda_score=neigh_lda.predict(x_test_lda) == y_test
    print("ACCURACY OF THE LDA MODEL: ", metrics.accuracy_score(y_test, neigh_lda.predict(x_test_lda)))
    # Baseline
    counter = Counter(y_train)
    most_common_element, occurrence_count = counter.most_common(1)[0]
    base_score = most_common_element == y_test

    # LDA vs Top2vec
    #Both are corect
    d1_n_11.append(sum(lda_score*top2vec_score))
    # LDA is correct Top is wrong
    d1_n_10.append(sum(lda_score*(1-top2vec_score)))
    # LDA is wrong Top is correct
    d1_n_01.append(sum((1-lda_score)*top2vec_score))
    #Both wrong
    d1_n_00.append(sum((1-lda_score)*(1-top2vec_score)))
    # difference in accuracy
    n=len(y_test)
    Etheta=(d1_n_10[i]-d1_n_01[i])/n
    if (n*(d1_n_10[i]+d1_n_01[i])-(d1_n_10[i]-d1_n_01[i])**2)!=0:
        Q=n**2*(n+1)*(Etheta+1)*(1-Etheta)/(n*(d1_n_10[i]+d1_n_01[i])-(d1_n_10[i]-d1_n_01[i])**2)
        f=(Etheta+1)/2*(Q-1)
        g=(1-Etheta)/2*(Q-1)
        # 95 % confidence interval
        p_u=2*beta.ppf(0.975,f,g)-1
        p_l=2*beta.ppf(0.025,f,g)-1
        print("LDA vs Top2Vec ")
        print("CI: ",p_l,":",p_u) 
    else:
        print("ZeroDivisionError")

    # P- value
    pval=2*binom.cdf(min(d1_n_10[i],d1_n_01[i]),(d1_n_10[i]+d1_n_01[i]),0.5)
    print("p-value: ", pval)

    #Sammenligniner med baseline

    #LDA vs Baseline
    #Both are corect
    d2_n_11.append(sum(lda_score*base_score))
    # LDA is correct Top is wrong
    d2_n_10.append(sum(lda_score*(1-base_score)))
    # LDA is wrong Top is correct
    d2_n_01.append(sum((1-lda_score)*base_score))
    #Both wrong
    d2_n_00.append(sum((1-lda_score)*(1-base_score)))
    # difference in accuracy
    n=len(y_test)
    Etheta=(d2_n_10[i]-d2_n_01[i])/n
    if (n*(d2_n_10[i]+d2_n_01[i])-(d2_n_10[i]-d2_n_01[i])**2)!=0:
        Q=n**2*(n+1)*(Etheta+1)*(1-Etheta)/(n*(d2_n_10[i]+d2_n_01[i])-(d2_n_10[i]-d2_n_01[i])**2)
        f=(Etheta+1)/2*(Q-1)
        g=(1-Etheta)/2*(Q-1)
        # 95 % confidence interval
        p_u=2*beta.ppf(0.975,f,g)-1
        p_l=2*beta.ppf(0.025,f,g)-1
        print("LDA vs baseline ")
        print("CI: ",p_l,":",p_u) 
    else:
        print("ZeroDivisionError")

    # P- value
    pval=2*binom.cdf(min(d2_n_10[i],d2_n_01[i]),(d2_n_10[i]+d2_n_01[i]),0.5)
    print("p-value: ", pval)

     #Top2vec vs Baseline
    #Both are corect
    d3_n_11.append(sum(top2vec_score*base_score))
    # LDA is correct Top is wrong
    d3_n_10.append(sum(top2vec_score*(1-base_score)))
    # LDA is wrong Top is correct
    d3_n_01.append(sum((1-top2vec_score)*base_score))
    #Both wrong
    d3_n_00.append(sum((1-top2vec_score)*(1-base_score)))
    # difference in accuracy
    n=len(y_test)
    Etheta=(d3_n_10[i]-d3_n_01[i])/n
    if (n*(d3_n_10[i]+d3_n_01[i])-(d3_n_10[i]-d3_n_01[i])**2)!=0:
        Q=n**2*(n+1)*(Etheta+1)*(1-Etheta)/(n*(d3_n_10[i]+d3_n_01[i])-(d3_n_10[i]-d3_n_01[i])**2)
        f=(Etheta+1)/2*(Q-1)
        g=(1-Etheta)/2*(Q-1)
        # 95 % confidence interval
        p_u=2*beta.ppf(0.975,f,g)-1
        p_l=2*beta.ppf(0.025,f,g)-1
        print("Top2vec vs baseline ")
        print("CI: ",p_l,":",p_u) 
    else:
        print("ZeroDivisionError")

    # P- value
    pval=2*binom.cdf(min(d3_n_10[i],d3_n_01[i]),(d3_n_10[i]+d3_n_01[i]),0.5)
    print("p-value: ", pval)






In [ ]:
d1_n_10

In [ ]:
d1_n_01

In [ ]:
d1_n_11

In [ ]:
d1_n_00

In [ ]:
n_neigh =[3,5,7,9,11,13]
x_train_top2vec, x_test_top2vec, y_train_top2vec, y_test_top2vec = train_test_split(feature_vectors_top2vec, labels, test_size = 0.3)
x_train_lda, x_test_lda, y_train_lda, y_test_lda = train_test_split(feature_vectors_lda, labels, test_size = 0.3)


for n in n_neigh:
    neigh1 = KNeighborsClassifier(n_neighbors=n)
    neigh1.fit(x_train_top2vec, y_train_top2vec)
    y_pred_knn = neigh1.predict(x_test_top2vec)
    print("ACCURACY OF top2vec MODEL with",n , " neighbors: ", metrics.accuracy_score(y_test_top2vec, y_pred_knn))

    neigh2 = KNeighborsClassifier(n_neighbors=n)
    neigh2.fit(x_train_lda, y_train_lda)
    y_pred_knn = neigh2.predict(x_test_lda)
    print("ACCURACY OF LDA MODEL with",n , " neighbors: ", metrics.accuracy_score(y_test_lda, y_pred_knn))